In [1]:
import numpy as np
from Ausgleichsbecken_class_file import Ausgleichsbecken_class
import matplotlib.pyplot as plt

#importing pressure conversion function
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion

In [2]:
L       = 1000.
n       = 10000                    # number of pipe segments in discretization
c       = 400. 
dx      = L/n                                   # length of each pipe segment
dt      = dx/c   

# # define constants
# initial_level               = 10.1        # m
# initial_influx              = 0.8       # m³/s
# conversion_pressure_unit    = 'mWS'

# area_base                   = 75.        # m²
# area_outflux                = (0.9/2)**2*np.pi       # m²
# critical_level_low          = 0.        # m
# critical_level_high         = 10.       # m
# simulation_timestep         = dt        # s

# # for while loop
# total_min_level             = 0.01      # m
# total_max_time              = 100     # s

# nt = int(total_max_time//simulation_timestep)

In [3]:
# define constants
initial_level               = 10.1        # m
initial_influx              = 1.       # m³/s
# initial_outflux             = 1.        # m³/s
# initial_pipeline_pressure   = 10.
# initial_pressure_unit       = 'mWS'
conversion_pressure_unit    = 'mWS'

area_base                   = 75.        # m²
area_outflux                = 2.       # m²
critical_level_low          = 0.        # m
critical_level_high         = 10.       # m
simulation_timestep         = dt    # s

# for while loop
total_min_level             = 0.01      # m
total_max_time              = 100     # s

nt = int(total_max_time//simulation_timestep)

In [4]:
%matplotlib qt

V = Ausgleichsbecken_class(area_base,area_outflux,critical_level_low,critical_level_high,simulation_timestep)
# V.set_initial_level(initial_level) 
# V.set_influx(initial_influx)
# V.set_outflux(initial_outflux)
# V.set_initial_pressure(pressure_conversion(initial_pipeline_pressure,input_unit = initial_pressure_unit, target_unit = 'Pa'),conversion_pressure_unit)
# V.pressure = converted_pressure
V.set_steady_state(initial_influx,initial_level,conversion_pressure_unit)

time_vec        = np.arange(0,nt+1,1)*simulation_timestep
outflux_vec     = np.zeros_like(time_vec)
outflux_vec[0]  = V.get_current_outflux()
level_vec       = np.zeros_like(time_vec)
level_vec[0]    = V.get_current_level()
pressure_vec    = np.zeros_like(time_vec)
pressure_vec[0] = V.get_current_pressure()

# pressure_vec    = np.full_like(time_vec,converted_pressure)*((np.sin(time_vec)+1)*np.exp(-time_vec/50))
 
i_max = -1

for i in range(1,nt+1):
    V.set_pressure(pressure_vec[i-1])
    V.set_outflux(outflux_vec[i-1])
    V.timestep_reservoir_evolution()
    outflux_vec[i] = V.get_current_outflux()
    level_vec[i]   = V.get_current_level()
    pressure_vec[i]   = V.get_current_pressure()
    if V.level < total_min_level:
        i_max = i
        break



In [5]:

fig1, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig1.set_figheight(10)
fig1.suptitle('Ausgleichsbecken')

ax1.plot(time_vec[:i_max],level_vec[:i_max], label='Water level')
ax1.set_ylabel(r'$h$ ['+V.level_unit+']')
ax1.set_xlabel(r'$t$ ['+V.time_unit+']')
ax1.legend()

ax2.plot(time_vec[:i_max],outflux_vec[:i_max], label='Outflux')
ax2.set_ylabel(r'$Q_{out}$ ['+V.flux_unit+']')
ax2.set_xlabel(r'$t$ ['+V.time_unit+']')
ax2.legend()

ax3.plot(time_vec[:i_max],pressure_conversion(pressure_vec[:i_max],'Pa',conversion_pressure_unit), label='Pipeline pressure at reservoir')
ax3.set_ylabel(r'$p_{pipeline}$ ['+conversion_pressure_unit+']')
ax3.set_xlabel(r'$t$ ['+V.time_unit+']')
ax3.legend()


fig1.tight_layout()            

In [6]:
V.get_current_level()

10.1